Set up test model class to open pipeline model from pkl file

In [2]:
import joblib

class TestModel:
    def predict(self, X):
        pipeline = joblib.load('pipeline.pkl')
        #print(pipeline)
        return pipeline.predict(X)

Recreate function from main file that is used for feature engineering in the pipeline.

In [4]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

class FeatureEngineering(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X = X.copy()
        X[['Salary', 'PreviousSalary']] = X[['Salary', 'PreviousSalary']].map(lambda x: x.replace('K', '000') if isinstance(x, str) else x)
        X[['Salary', 'PreviousSalary']] = X[['Salary', 'PreviousSalary']].astype(int)
        X['Raise'] = (X['Salary'] / X['PreviousSalary']) - 1
        X['ReviewDiff'] = X['SelfReview'] - X['SupervisorReview']
        X['ReviewOverRaise'] = X['SelfReview'] / (X['Raise'] +.0001)
        X['LongDist'] = np.where(X['Distance'].isin(['~20miles', '>30miles']), 1, 0)
        X['HighStress'] = np.where(X['StressLevel'] > 3.5, 1, 0)
        
        return X
    

In [5]:
# In the second notebook
import pandas as pd

from sklearn.metrics import classification_report

def production(X_path, y_path):
    # load model
    model = TestModel()
    
    # load data
    df_X = pd.read_csv(X_path)

    # make the changes if required 
    # -------------------------


    # -------------------------
    pred = model.predict(df_X)

    df_y = pd.read_csv(y_path)['Left']
    print(classification_report(df_y, pred))
    

production( 
    X_path='https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/final/employee_departure_dataset_X.csv',
    y_path='https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/final/employee_departure_dataset_y.csv'
)
   

              precision    recall  f1-score   support

           0       0.94      0.88      0.91    319539
           1       0.81      0.90      0.85    180461

    accuracy                           0.89    500000
   macro avg       0.87      0.89      0.88    500000
weighted avg       0.89      0.89      0.89    500000

